## K-Lime & Shapley Scoring Pipeline

scoring-pipeline/example.py : 予測値のスコアリングとShapleyのスコアリングに関するサンプルプログラム  
scoring-pipeline-mli/run_example.py : K-Limeのスコアリングに関するサンプルプログラム 

In [1]:
import pandas as pd
import numpy as np
from numpy import nan
from scipy.special._ufuncs import expit

### K-Lime Scoring

In [2]:
from scoring_mli_experiment_eb096b2c_4fcd_11eb_9924_0242ac110002 import KLimeScorer

2021-01-06 07:30:39,906 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DEBUG  | Registered SegFault library for signals 'all'
2021-01-06 07:30:40,403 C: NA  D:  NA    M:  NA    NODE:SERVER      29712  DEBUG  | Could not locate cudart (None) or other error: #_/home/ubuntu/model-deployment/common/hydraulic/python-scoring-pipeline/mli_1906/scoring-pipeline-mli/env/bin/python -m ipykern___uncher -f /home/ubuntu/.local/share/jupyter/runtime/kernel-c0c61627-bc97-4e53-8908-10a28c549bc4.jsonget_cuda_versions_subprocess: undefined symbol: cudaRuntimeGetVersion
2021-01-06 07:30:40,430 C: NA  D:  NA    M:  NA    NODE:SERVER      29712  DEBUG  | Could not locate cudnn (None) or other error #_/home/ubuntu/model-deployment/common/hydraulic/python-scoring-pipeline/mli_1906/scoring-pipeline-mli/env/bin/python -m ipykern___uncher -f /home/ubuntu/.local/share/jupyter/runtime/kernel-c0c61627-bc97-4e53-8908-10a28c549bc4.jsonget_cuda_versions_subprocess: undefined symbol: cudnnGetVersion


In [3]:
mli_scorer = KLimeScorer()

2021-01-06 07:30:43,184 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | License manager initialized
2021-01-06 07:30:43,185 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | -----------------------------------------------------------------
2021-01-06 07:30:43,186 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | Checking whether we have a valid license...
2021-01-06 07:30:43,186 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | No Cloud provider found
2021-01-06 07:30:43,187 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | License inherited from environment
2021-01-06 07:30:43,194 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | 
2021-01-06 07:30:43,195 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | license_version:1
2021-01-06 07:30:43,196 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | serial_number:3
2021-01-06 07:30:43,196 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | licensee_org

In [4]:
mli_scorer.score_reason_codes([
    '-2.7997',  # x3
    '-2.4307',  # x1
    '-2.3697',  # x4
])

[-0.39610478260245907,
 -0.5276176194651054,
 3.415107055976632,
 1.233942981112774]

In [5]:
mli_scorer.score_reason_codes([
    '0',  # x3
    '4.8',  # x1
    '-6.3',  # x4
])

[-0.0, 5.08845501846021, 2.1897282740067365, 0.5037904019017483]

In [6]:
columns = [
    pd.Series(['-2.7997', '-2.5746', '-2.6275', '-2.7208', '-2.4735'], name='x3', dtype='float64'),
    pd.Series(['-2.4307', '-2.9838', '-2.4492', '-2.4307', '-2.4871'], name='x1', dtype='float64'),
    pd.Series(['-2.3697', '-3.4233', '-2.4827', '-3.4233', '-2.3194'], name='x4', dtype='float64'),
]
df = pd.concat(columns, axis=1)
print(mli_scorer.score_reason_codes_batch(df))

         x3        x1        x4  Intercept
0 -0.396105 -0.527618  3.415107   1.233943
1  2.075710 -1.466986  0.969845   0.896459
2 -0.371742 -0.531633  3.577958   1.233943
3  2.193580 -1.195055  0.969845   0.896459
4 -0.349954 -0.539860  3.342617   1.233943


In [7]:
import datatable as dt
#dt.Frame(df).to_csv("test.csv")          # turn above dummy frame into a CSV (for convenience)
#test_dt = dt.fread("test.csv", na_strings=['', '?', 'None', 'nan', 'NA', 'N/A', 'unknown', 'inf', '-inf', '1.7976931348623157e+308', '-1.7976931348623157e+308'])           # parse test set CSV file into datatable (with consistent NA handling)
test_dt = dt.Frame(df)
preds_df = mli_scorer.score_reason_codes_batch(test_dt)   # make predictions (pandas frame)
#dt.Frame(preds_df).to_csv("preds.csv")   # save pandas frame to CSV using datatable

In [8]:
dt.Frame(preds_df)

,x3,x1,x4,Intercept
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,−0.396105,−0.527618,3.41511,1.23394
1,2.07571,−1.46699,0.969845,0.896459
2,−0.371742,−0.531633,3.57796,1.23394
3,2.19358,−1.19505,0.969845,0.896459
4,−0.349954,−0.53986,3.34262,1.23394


In [9]:
mli_scorer.quantile_bin_row([
    '-2.7997',  # x3
    '-2.4307',  # x1
    '-2.3697',  # x4
])

['-2.7997', '-2.4307', '-2.3697']

In [10]:
!pwd

/home/ubuntu/Driverless_AI_ScoringPipelines/Python_MLI_Scoring/Local


In [12]:
import pandas
THRESHOLD = 0.2  # the max absolute difference between KLime model score and DAI score where DAI reason codes should be reported
#exec(open("../scoring-pipeline/example.py").read())
#exec(open("example.py").read())

In [13]:
#Example DataFrame
mli_df = pandas.DataFrame(pandas.np.array([[
    '-2.7997',
    '-2.4307',
    '-2.3697',
]]), columns=mli_scorer.get_column_names())

mli_df

,x3,x1,x4
0,-2.7997,-2.4307,-2.3697


### Shapley Scoring

In [15]:
from scoring_h2oai_experiment_3a80fcea_4fcb_11eb_9924_0242ac110002 import Scorer

2021-01-06 07:33:12,549 C:  0% D:202.9GB M:30.5GB  NODE:SERVER      29644  INFO   | font search path ['/home/ubuntu/model-deployment/common/hydraulic/python-scoring-pipeline/mli_1906/scoring-pipeline-mli/env/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf', '/home/ubuntu/model-deployment/common/hydraulic/python-scoring-pipeline/mli_1906/scoring-pipeline-mli/env/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/afm', '/home/ubuntu/model-deployment/common/hydraulic/python-scoring-pipeline/mli_1906/scoring-pipeline-mli/env/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2021-01-06 07:33:12,770 C:  0% D:202.9GB M:30.5GB  NODE:SERVER      29644  INFO   | generated new fontManager


In [17]:
scorer = Scorer()

2021-01-06 07:33:27,675 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | Creating custom transformers directory.
2021-01-06 07:33:27,681 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | Creating custom models directory.
2021-01-06 07:33:27,716 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | Creating custom scorers directory.
2021-01-06 07:33:27,719 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  INFO   | Creating custom data directory.
2021-01-06 07:33:27,732 C:  1% D:202.9GB M:30.5GB  NODE:SERVER      29644  INFO   | Starting H2O server for recipes.  url: None, ip: None, port: 50351, name: DAI-H2O-RECIPES-1.9.0., threads: 8
Checking whether there is an H2O instance running at http://localhost:50351 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_275"; OpenJDK Runtime Environment (build 1.8.0_275-8u275-b01-0ubuntu1~18.04-b01); OpenJDK 64-Bit Server VM (build 25.275-b01, mixed mode)
  Starting server 

H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,7 months and 23 days !!!
H2O_cluster_name:,DAI-H2O-RECIPES-1.9.0.
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.971 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


2021-01-06 07:33:32,318 C: 14% D:202.9GB M:30.3GB  NODE:SERVER      29644  INFO   | RECIPE H2O-3 server started
2021-01-06 07:33:32,320 C: 14% D:202.9GB M:30.3GB  NODE:SERVER      29644  INFO   | Started H2O version 3.30.0.3 at http://127.0.0.1:50351


In [29]:
scorer

In [18]:
# Make the row compatible with DAI model input
dai_df = mli_df.reindex(scorer.get_column_names(), axis=1)
dai_df

,x1,x3,x4
0,-2.4307,-2.7997,-2.3697


In [19]:
# Get the DAI model score
dai_score = scorer.score_batch(dai_df)
dai_score

2021-01-06 07:33:36,040 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DATA   | Warning: column <<<'x1'>>> is of type str, but should be float32. Please check input data.
2021-01-06 07:33:36,046 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DATA   | Warning: column <<<'x3'>>> is of type str, but should be float32. Please check input data.
2021-01-06 07:33:36,050 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DATA   | Warning: column <<<'x4'>>> is of type str, but should be float32. Please check input data.
2021-01-06 07:33:36,264 C:  4% D:202.9GB M:30.3GB  NODE:SERVER      29644  INFO   | Submitted    0 and Completed    0 non-identity feature engineering tasks out of    3 total tasks (including    3 identity)


,y,y.lower,y.upper
0,2.633075,0.259097,5.167462


In [20]:
type(scorer)

scoring_h2oai_experiment_3a80fcea_4fcb_11eb_9924_0242ac110002.scorer.Scorer

In [21]:
# Get the prediction contributions (Shapley reason codes) from DAI model
dai_reason_codes = scorer.score_batch(dai_df, pred_contribs=True)
dai_reason_codes

2021-01-06 07:33:39,287 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DATA   | Warning: column <<<'x1'>>> is of type str, but should be float32. Please check input data.
2021-01-06 07:33:39,291 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DATA   | Warning: column <<<'x3'>>> is of type str, but should be float32. Please check input data.
2021-01-06 07:33:39,295 C: NA  D:  NA    M:  NA    NODE:SERVER      29644  DATA   | Warning: column <<<'x4'>>> is of type str, but should be float32. Please check input data.
2021-01-06 07:33:39,511 C:  2% D:202.9GB M:30.3GB  NODE:SERVER      29644  INFO   | Submitted    0 and Completed    0 non-identity feature engineering tasks out of    3 total tasks (including    3 identity)


,contrib_0_x1,contrib_2_x3,contrib_3_x4,contrib_bias
0,-58650.125,95974.257812,72283.265625,466291.90625


In [22]:
# Get KLime reason codes
klime_reason_codes = mli_scorer.score_reason_codes_batch(mli_df)
klime_reason_codes

,x3,x1,x4,Intercept
0,-0.396105,-0.527618,3.415107,1.233943


In [23]:
# KLime model score is the sum of all the reason codes+intercept (included in the output of score_reason_codes* methods)
klime_score = mli_scorer.score_reason_codes_batch(mli_df).sum(axis=1)
klime_score

0    3.725328
dtype: float64

In [24]:
mli_scorer.score_reason_codes_batch(mli_df)

,x3,x1,x4,Intercept
0,-0.396105,-0.527618,3.415107,1.233943


In [25]:
type(mli_scorer)

scoring_mli_experiment_eb096b2c_4fcd_11eb_9924_0242ac110002.klime_scorer.KLimeScorer

In [26]:
# If the difference between KLime and the DAI model is greater than a threshold
# then use the reason codes from the actual DAI (in transformed feature space)
if abs(dai_score.values[0][0] - klime_score.values[0]) > THRESHOLD:
    print(" KLIME score vs DAI score difference is higher than threshold. Reporting Reason codes from DAI model:")
    print(str(dai_reason_codes))
else:
    print(" KLIME score within threshold difference of DAI model. Reporting Reason codes from KLIME:")
    print(str(klime_reason_codes))

 KLIME score vs DAI score difference is higher than threshold. Reporting Reason codes from DAI model:
   contrib_0_x1  contrib_2_x3  contrib_3_x4  contrib_bias
0    -58650.125  95974.257812  72283.265625  466291.90625
